In [ ]:
# MQTT client

In [ ]:
import paho.mqtt.client as mqtt

client = mqtt.Client()

client.connect("192.168.0.151", 1883, 60)

# Import pose estimation model

## Define output format

Let's load the JSON file which describes the human pose task.  This is in COCO format, it is the category descriptor pulled from the annotations file.  We modify the COCO category slightly, to add a neck keypoint.  We will use this task description JSON to create a topology tensor, which is an intermediate data structure that describes the part linkages, as well as which channels in the part affinity field each linkage corresponds to.

In [6]:
import os
os.environ['MPLCONFIGDIR'] = os.getcwd() + "/configs/" # Specify MatplotLib config folder

import json
import numpy as np
# Requiere https://github.com/NVIDIA-AI-IOT/trt_pose
import trt_pose.coco
from trt_pose.draw_objects import DrawObjects
from trt_pose.parse_objects import ParseObjects

with open('human_pose.json', 'r') as f:
    human_pose = json.load(f)

topology = trt_pose.coco.coco_category_to_topology(human_pose)

parse_objects = ParseObjects(topology)
draw_objects = DrawObjects(topology)

## Import TensorRT optimized model

Next, we'll load our model. It has been optimized using another Notebook and saved so that we do not need to perform optimization again, we can just load the model. Please note that TensorRT has device specific optimizations, so you can only use an optimized model on similar platforms.

In [2]:
import torch
# Requiere https://github.com/NVIDIA-AI-IOT/torch2trt
from torch2trt import TRTModule

OPTIMIZED_MODEL = 'resnet18_baseline_att_224x224_A_epoch_249_trt.pth'

model_trt = TRTModule()
model_trt.load_state_dict(torch.load(OPTIMIZED_MODEL))

<All keys matched successfully>

# Define video-processing pipeline

## Pre-process image for TRT_Pose

Next, let's define a function that will preprocess the image, which is originally in BGR8 / HWC format. It is formated to the default Torch format.

In [3]:
import cv2
import torchvision.transforms as transforms
import PIL.Image

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()
device = torch.device('cuda')

def preprocess(image):
    global device
    device = torch.device('cuda')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

## Access video feed

Access images streamed by a WiFi camera on the local network.

In [4]:
WIDTH, HEIGHT = 224, 224 # Imposed by the model

import ipywidgets
from IPython.display import display
import urllib.request

image_w = ipywidgets.Image(format='jpeg')

display(image_w)

url_esp32 = 'http://192.168.0.162/capture'
def fetch_image(url):
    imgResp = urllib.request.urlopen(url)
    imgNp = np.array(bytearray(imgResp.read()),dtype=np.uint8)
    img = cv2.imdecode(imgNp,-1)
    return cv2.resize(img, (WIDTH, HEIGHT), interpolation = cv2.INTER_AREA)

Image(value=b'', format='jpeg')

In [ ]:
## Get keypoints from the peaks 

In [ ]:
def get_keypoint(objects, index, peaks):
    kpoint = []
    human = objects[0][index]
    C = human.shape[0]
    for j in range(C):
        k = int(human[j])
        if k >= 0:
            peak = peaks[0][j][k]   # peak[1]:width, peak[0]:height
            peak = (j, float(peak[0]), float(peak[1]))
            kpoint.append(peak)
            print('index:%d : success [%5.3f, %5.3f]'%(j, peak[1], peak[2]) )
        else:    
            peak = (j, None, None)
            kpoint.append(peak)
            print('index:%d : None'%(j) )
    return kpoint

In [ ]:
## Detect if one hand is raised


In [ ]:
def isRaisingHand(poseKeypoints):
    # TODO : transform counts, objects and peaks into keypoints, Also find the right keypoints
        raisingLeft = False
        raisingRight = False
        rightHand_idx, rightHand_y, rightHand_x = poseKeypoints[10]
        leftHand_idx, leftHand_y, leftHand_x = poseKeypoints[9]
        rightShoulder_idx, rightShoulder_y, rightShoulder_x = poseKeypoints[6]
        leftShoulder_idx, leftShoulder_y, leftShoulder_x = poseKeypoints[5]
        try:
            shoulderSlope = (rightShoulder_y - leftShoulder_y) / (
                rightShoulder_x - leftShoulder_x
            )
        except:
            shoulderSlope = 0.0
        shoulderOri = rightShoulder_y - shoulderSlope * rightShoulder_x

        if leftHand_a > 0.1:
            raisingLeft = leftHand_y > (
                shoulderSlope * leftHand_x + shoulderOri
            )  # y axis oriented from top to down in images
            raisingLeft = (
                raisingLeft and leftHand_y > poseKeypoints[6, 1]
            )  # Check if hand above elbow
        else:
            raisingLeft= False

        if rightHand_a > 0.1:
            raisingRight = rightHand_y > (shoulderSlope * rightHand_x + shoulderOri)
            raisingRight = raisingRight and rightHand_y > poseKeypoints[3, 1]
        else:
            raisingRight = False
        return raisingRight or raisingLeft

In [ ]:
## Processing loop

The *execute()* function contains the whole analysis process: 
- Read image
- Pre-process to Torch format
- Infere key-points
- Draw skeleton on the input image
- Update in output window.

In [10]:
try:
    while True:
        image = fetch_image(url_esp32)
        data = preprocess(image)
        cmap, paf = model_trt(data)
        cmap, paf = cmap.detach().cpu(), paf.detach().cpu()
        counts, objects, peaks = parse_objects(cmap, paf) #, cmap_threshold=0.15, link_threshold=0.15)
        for i in range(counts[0]):
            print("Human index:%d "%( i ))
            keypoints = get_keypoint(objects, i, peaks)
            if isRaisingHand(keypoints):
                client.publish("lamp", "2")
        draw_objects(image, counts, objects, peaks)
        image_w.value = bytes(cv2.imencode('.jpg', image[:, ::-1, :])[1])
except KeyboardInterrupt:
    print('Video processing stopped')

Video processing stopped
